In [11]:
!pip install numpy matplotlib

In [12]:
###### Importación librerías ######

import numpy as np
import matplotlib.pyplot as plt

# 1. Carga y exploración de datos

Gráfico de conexiones
![grafo_conexiones]()

In [13]:
# Cargar datos del grafo
conexiones = np.load("conexiones.npy")
print(conexiones)

estaciones = np.load("estaciones.npy", allow_pickle=True)
print(estaciones)

[('E1', 'E2') ('E1', 'E3') ('E1', 'E4') ('E1', 'E12') ('E1', 'E13')
 ('E1', 'E14') ('E2', 'E5') ('E2', 'E6') ('E2', 'E7') ('E3', 'E7')
 ('E3', 'E8') ('E3', 'E5') ('E4', 'E9') ('E4', 'E10') ('E4', 'E6')
 ('E5', 'E8') ('E5', 'E11') ('E6', 'E9') ('E6', 'E11') ('E7', 'E10')
 ('E7', 'E11') ('E8', 'E10') ('E9', 'E11') ('E12', 'E15') ('E12', 'E16')
 ('E12', 'E17') ('E13', 'E17') ('E13', 'E18') ('E13', 'E15')
 ('E14', 'E19') ('E14', 'E20') ('E14', 'E16') ('E15', 'E18')
 ('E15', 'E21') ('E16', 'E19') ('E16', 'E22') ('E17', 'E20')
 ('E17', 'E21') ('E18', 'E22') ('E19', 'E22') ('E20', 'E21') ('E8', 'E23')
 ('E10', 'E23') ('E11', 'E23') ('E20', 'E23') ('E21', 'E23')
 ('E22', 'E23')]
[('E1', list(['temperatura', 'sanitaria']),  0. )
 ('E2', list(['temperatura']), 12.4) ('E3', list(['temperatura']),  9.8)
 ('E4', list(['temperatura']), 14.2) ('E5', list(['temperatura']), 10.5)
 ('E6', list(['temperatura']),  8.9) ('E7', list(['temperatura']), 13.1)
 ('E8', list(['temperatura']), 11.7) ('E9', list(['

# 2. Formulación formal del problema

Un par de productos se deben mover desde un punto a otro gastando la menor cantidad de recursos. Para lograrlo el producto debe atravesar diferentes estaciones. Cada estación tiene un costo asociado al traslado, y una restricción que indica si recibe uno u otro producto.

Se debe hallar un cámino que cumpla lo siguiente:

**MIN f(producto, nodo_origen, nodo_destino) = sum(costo(nodo_origen, nodo_next))** , suma desde nodo_origen hasta nodo_destino donde el valor a sumar corresponde al valor del enlace.

s.a
1. nodo_origen = E1
2. nodo_destino = E23
3. producto medico debe pasar solo por estaciones de temperatura
4. producto alimenticio debe pasar solo por estaciones de sanidad

# 3. Implementación de los tres algoritmos

## DFS Algorithm

Algoritmo de búsqueda no informado. Se implementará con lógica de búsqueda In Order. Este algoritmo podría ser bueno para este problema dado que es pequeño (menos de 50 nodos)

In [14]:
# Implementación DFS
# conexiones
# estaciones

# Estructura Nodo
class Nodo():
    def __init__(self, estacion:str, certificado:list, costo:float):
        self.estacion = estacion
        self.certificado = certificado
        self.costo = costo
        self.vecinos = [str(nodo[1]) for nodo in conexiones if nodo[0] == self.estacion]

def generar_nodo(estacion):
    info = [info_estacion for info_estacion in estaciones if info_estacion[0] == estacion][0]
    return Nodo(
        estacion=estacion,
        certificado=info[1],
        costo=info[2]
    )


def DFS(certificado:str, nodo_origen:str, nodo_destino:str, acc:float=0, path=None):
    '''
    Algoritmo Deep First Search implementado con lógica in order, es decir,
    itera tomando el primer hijo de cada nodo según el orden en el que se
    hayan insertado en el árbol/grafo. Retorna el primer camino que encuentra.

    Parámetros:
    certificado (str): "sanitaria" o "temperatura".
    nodo_origen (str): nombre de la estación, ej: "E1"
    nodo_destino (str): nombre de la estación, ej: "E23"
    acc (float): costo acumulado del path
    path (list<str>): lista con los nombres de las estaciones

    Retorna:
    (path, acc) (tuple): Si encuentra un cámino desde nodo_origen a nodo_destino
    None: Si no encuentra un cámino desde nodo_origen a nodo_destino

    '''

    # Validaciones #
    if certificado not in ["sanitaria", "temperatura"]:
        print("Error: tipo de certificado no válido")
        return None

    if not nodo_origen:
        print("Nodo inválido o Ruta no encontrada")
        return None

    # Inicialización #
    if path is None:
        path = []

    nodo = generar_nodo(nodo_origen)

    # Condición base #
    if nodo.estacion == nodo_destino:
        path.append(nodo.estacion)
        acc+=nodo.costo
        return path, acc


    # Recursividad #
    if certificado in nodo.certificado and nodo.estacion not in path:
        path.append(nodo.estacion)
        acc+=nodo.costo
        for vecino in nodo.vecinos:
            resultado = DFS(certificado, vecino, nodo_destino, acc, path.copy())
            if resultado:
                return resultado

    return None

# Aplicación del algoritmo
dfs_path_sanitaria = DFS("sanitaria", "E1", "E23")
dfs_path_temperatura = DFS("temperatura", "E1", "E23")
print(dfs_path_sanitaria)
print(dfs_path_temperatura)


# Validación
def validate_path(path:list, cost:float):
    acc = 0
    for i in range(len(path)-1):
        nodo_actual = generar_nodo(path[i])
        vecinos = nodo_actual.vecinos
        acc+= nodo_actual.costo
        if path[i+1] not in vecinos:
            print(f"Error: no es posible llegar de {path[i]} a {path[i+1]}")
            return False
    acc+= generar_nodo(path[i+1]).costo
    return True and acc == cost

print(f"El path es {validate_path(dfs_path_sanitaria[0], dfs_path_sanitaria[1])}")
print(f"El path es {validate_path(dfs_path_temperatura[0], dfs_path_temperatura[1])}")



(['E1', 'E12', 'E15', 'E18', 'E22', 'E23'], np.float32(38.300003))
(['E1', 'E2', 'E5', 'E8', 'E10', 'E23'], np.float32(49.399998))
El path es True
El path es True


# UCS

Funciona priorizando las rutas con el menor costo total desde el punto de partida.

In [15]:
# conexiones
# estaciones


def UCS(certificado:str, nodo_origen:str, nodo_destino:str):
    '''
    Algoritmo Uniform Cost Search usa una frontera ordenada por costo acumulado.

    Parámetros:
    certificado (str): "sanitaria" o "temperatura".
    nodo_origen (str): nombre de la estación, ej: "E1"
    nodo_destino (str): nombre de la estación, ej: "E23"

    Retorna:
    (path, acc) (tuple): Si encuentra un camino desde nodo_origen a nodo_destino
    None: Si no encuentra un camino desde nodo_origen a nodo_destino
    '''

    if certificado not in ["sanitaria", "temperatura"]:
        print("Error: tipo de certificado no válido")
        return None

    if not nodo_origen:
        print("Nodo inválido o Ruta no encontrada")
        return None

    # Inicialización
    # frontera: lista de tuplas (costo_acumulado, path)
    frontera = [(0.0, [nodo_origen])]
    explorados = set()
    mejor_costo = {}  # mejor costo conocido para cada estación

    while frontera:
        # Ordenar frontera por costo acumulado
        frontera.sort(key=lambda x: x[0])
        costo_actual, path_actual = frontera.pop(0)

        estacion_actual = path_actual[-1]

        if estacion_actual in explorados:
            continue

        # Condicion Base
        if estacion_actual == nodo_destino:
            return path_actual, costo_actual

        explorados.add(estacion_actual)

        nodo_actual = generar_nodo(estacion_actual)

        if certificado in nodo_actual.certificado:
            for vecino in nodo_actual.vecinos:
                if vecino not in explorados:
                    nodo_vecino = generar_nodo(vecino)

                    if certificado in nodo_vecino.certificado:
                        nuevo_costo = costo_actual + nodo_vecino.costo
                        nuevo_path = path_actual + [vecino]

                        # Verificar si es mejor que el costo conocido
                        if vecino not in mejor_costo or nuevo_costo < mejor_costo[vecino]:
                            mejor_costo[vecino] = nuevo_costo
                            frontera.append((nuevo_costo, nuevo_path))

    return None

# Aplicación del algoritmo
ucs_path_sanitaria = UCS("sanitaria", "E1", "E23")
ucs_path_temperatura = UCS("temperatura", "E1", "E23")
print("UCS Sanitaria:", ucs_path_sanitaria)
print("UCS Temperatura:", ucs_path_temperatura)

# Validación
print(f"UCS Sanitaria - El path es válido: {validate_path(ucs_path_sanitaria[0], ucs_path_sanitaria[1])}")
print(f"UCS Temperatura - El path es válido: {validate_path(ucs_path_temperatura[0], ucs_path_temperatura[1])}")

UCS Sanitaria: (['E1', 'E14', 'E20', 'E23'], np.float32(19.0))
UCS Temperatura: (['E1', 'E3', 'E8', 'E23'], np.float32(21.5))
UCS Sanitaria - El path es válido: True
UCS Temperatura - El path es válido: True



##BFS Algorithm
Es un algoritmo de busqueda no informado que va explorando nodos vecinos hasta llegar a un estado objetivo,que tan lejos este el objetivo fe estado inicial puede ser una herramienta util.


In [16]:


class Nodo():
    def __init__(self, estacion:str, certificado:list, costo:float):
        self.estacion = estacion
        self.certificado = certificado
        self.costo = costo
        #Encuetra los vecinos desde 'conexiones' array
        self.vecinos = [str(nodo[1]) for nodo in conexiones if nodo[0] == self.estacion]

# --- generar_nodo funcion
def generar_nodo(estacion):
    # Encontrar la siguiente estacion en 'estaciones' array
    info = [info_estacion for info_estacion in estaciones if info_estacion[0] == estacion]
    if not info:
        raise ValueError(f"Station '{estacion}' not found in 'estaciones' data.")
    info = info[0] #Tomar la primera entrada valida
    return Nodo(
        estacion=estacion,
        certificado=info[1],
        costo=info[2]
    )

# BFS
def BFS(certificado_req:str, id_origen:str, id_destino:str):
    '''
    Algoritmo Breadth-First Search implementado con una cola (lista de Python).
    Encuentra el camino más corto en términos de número de aristas.

    Parámetros:
    certificado_req (str): "sanitaria" o "temperatura".
    id_origen (str): nombre de la estación, ej: "E1"
    id_destino (str): nombre de la estación, ej: "E23"

    Retorna:
    (camino, costo_total) (tuple): Si encuentra un camino desde id_origen a id_destino
    None: Si no encuentra un camino desde id_origen a id_destino
    '''

    # Validaciones #
    if certificado_req not in ["sanitaria", "temperatura"]:
        print("Error: tipo de certificado no válido")
        return None

    # Cola: Almacena tuples (current_node_name, current_path_list, accumulated_cost)
    # Usando una lista de python como cola (pop(0) is O(N))
    cola = []

    # para evitar que el algorito revisite los nodos
    visitados = set()

    # Etapa inicial
    nodo_inicial_obj = generar_nodo(id_origen)

    # Certificar si los nodos cumple las certificaiones
    # O si es E1 O E23, que siempre la cumplen
    es_estacion_especial = (nodo_inicial_obj.estacion == 'E1' or nodo_inicial_obj.estacion == 'E23')
    if not (es_estacion_especial or certificado_req in nodo_inicial_obj.certificado):
        print(f"Error: Nodo origen '{id_origen}' no cumple con el certificado '{certificado_req}'.")
        return None

    cola.append((nodo_inicial_obj.estacion, [nodo_inicial_obj.estacion], nodo_inicial_obj.costo))
    visitados.add(nodo_inicial_obj.estacion)

    while cola:
        nombre_estacion_actual, camino_actual, costo_actual = cola.pop(0) # Dequeue operation

        if nombre_estacion_actual == id_destino:
            return camino_actual, costo_actual


        nodo_actual_obj = generar_nodo(nombre_estacion_actual)

        # explorar vecinos
        for nombre_vecino in nodo_actual_obj.vecinos:
            if nombre_vecino not in visitados:
                nodo_vecino_obj = generar_nodo(nombre_vecino)

                # Verificar los requisitos de certificación para el vecino.
                # E1 y E23 tiene verificacion
                es_vecino_especial = (nodo_vecino_obj.estacion == 'E1' or nodo_vecino_obj.estacion == 'E23')

                if es_vecino_especial or certificado_req in nodo_vecino_obj.certificado:
                    visitados.add(nombre_vecino)
                    nuevo_camino = camino_actual + [nombre_vecino]
                    nuevo_costo = costo_actual + nodo_vecino_obj.costo
                    cola.append((nombre_vecino, nuevo_camino, nuevo_costo))

    return None

# --- Aplicacion del algoritmo
camino_bfs_sanitaria, costo_bfs_sanitaria = BFS("sanitaria", "E1", "E23")
camino_bfs_temperatura, costo_bfs_temperatura = BFS("temperatura", "E1", "E23")

print(f"BFS Path Sanitaria: {camino_bfs_sanitaria}, Cost: {costo_bfs_sanitaria}")
print(f"BFS Path Temperatura: {camino_bfs_temperatura}, Cost: {costo_bfs_temperatura}")

# Funcion de validacion
def validate_path(camino:list, costo:float):
    if camino is None:
        return False

    costo_calculado = 0.0
    #clcular costo
    info_primer_nodo = [info_estacion for info_estacion in estaciones if info_estacion[0] == camino[0]][0]
    costo_calculado += info_primer_nodo[2]

    for i in range(len(camino)-1):
        nombre_nodo_actual = camino[i]
        nombre_siguiente_nodo = camino[i+1]

        nodo_actual_obj = generar_nodo(nombre_nodo_actual)

        # Chequear si el nodo vecino es valido
        if nombre_siguiente_nodo not in nodo_actual_obj.vecinos:
            print(f"Error: no es posible llegar de {nombre_nodo_actual} a {nombre_siguiente_nodo}")
            return False

    # Agregar costo del siguiente nodo (según la descripción del problema, el costo se aplica cuando el producto pasa)
    # El problema indica "Cada vez que un producto pasa por una estación, se aplica el costo de procesamiento de dicha estación."
    # Esto implica que el costo de la estación *actual* se aplica al entrar en ella.
    # El BFS acumula el costo del *siguiente* nodo al agregarlo a la ruta.
    # Por lo tanto, la validación debe sumar los costos de todas las estaciones en la ruta.

    # Re-calcular`costo_calculado` :
    costo_calculado = 0.0
    for nombre_estacion_en_camino in camino:
        info_estacion = [info_estacion for info_estacion in estaciones if info_estacion[0] == nombre_estacion_en_camino][0]
        costo_calculado += info_estacion[2]

    return (costo_calculado == costo)


print(f"BFS Path Sanitaria is valid: {validate_path(camino_bfs_sanitaria, costo_bfs_sanitaria)}")
print(f"BFS Path Temperatura is valid: {validate_path(camino_bfs_temperatura, costo_bfs_temperatura)}")

BFS Path Sanitaria: ['E1', 'E14', 'E20', 'E23'], Cost: 19.0
BFS Path Temperatura: ['E1', 'E3', 'E8', 'E23'], Cost: 21.5
BFS Path Sanitaria is valid: True
BFS Path Temperatura is valid: True


# 4. Función de visualización de la solución

## DFS

In [17]:
### Funciona solo para el DFS ###
def solucion_interactiva(path:list, tipo_certificado:str):
    if tipo_certificado not in ["s", "t"]:
        print("Error: ingresa certificado en formato 's' sanitaria o 't' temperatura")
        return None
    acc = 0
    for i in range(len(path)-1):
        nodo_actual = generar_nodo(path[i])
        certificados_actual = ', '.join([c[0] for c in nodo_actual.certificado])
        vecinos = nodo_actual.vecinos
        acc+= nodo_actual.costo
        found_next_node = 0
        print("=============================")
        print(f"\033[94m Iteración: {i} \033[0m")
        print(f"\033[92mEstación: ({nodo_actual.estacion}, ({certificados_actual}), {nodo_actual.costo:.2f})\033[0m")
        print(f"\033[96mPath: {path[0:i+1]} - Costo Acc: {acc:.2f}")
        print("\033[95mHijos: ", end="")
        for estacion in vecinos:
            estacion_info = [est for est in estaciones if est[0] == estacion][0]
            certificados = ', '.join([c[0] for c in estacion_info[1]])
            if tipo_certificado in certificados:
                found_next_node+=1
            if found_next_node == 1:
                print(f"\033[93m({estacion_info[0]}, ({certificados}), {estacion_info[2]:.2f}) ", end="")
            else:
                print(f"\033[95m({estacion_info[0]}, ({certificados}), {estacion_info[2]:.2f}) ", end="")
        print("\033[0m")

print("_______________________________ Solución para producto Sanitario _______________________________")
solucion_interactiva(dfs_path_sanitaria[0], "s")
print("_______________________________ Solución para producto Temperatura _______________________________")
solucion_interactiva(dfs_path_temperatura[0], "t")

_______________________________ Solución para producto Sanitario _______________________________
 Iteración: 0 
Estación: (E1, (t, s), 0.00)
Path: ['E1'] - Costo Acc: 0.00
Hijos: (E2, (t), 12.40) (E3, (t), 9.80) (E4, (t), 14.20) (E12, (s), 8.70) (E13, (s), 11.30) (E14, (s), 9.10) 
 Iteración: 1 
Estación: (E12, (s), 8.70)
Path: ['E1', 'E12'] - Costo Acc: 8.70
Hijos: (E15, (s), 7.50) (E16, (s), 10.80) (E17, (s), 6.90) 
 Iteración: 2 
Estación: (E15, (s), 7.50)
Path: ['E1', 'E12', 'E15'] - Costo Acc: 16.20
Hijos: (E18, (s), 11.70) (E21, (s), 7.20) 
 Iteración: 3 
Estación: (E18, (s), 11.70)
Path: ['E1', 'E12', 'E15', 'E18'] - Costo Acc: 27.90
Hijos: (E22, (s), 10.40) 
 Iteración: 4 
Estación: (E22, (s), 10.40)
Path: ['E1', 'E12', 'E15', 'E18', 'E22'] - Costo Acc: 38.30
Hijos: (E23, (t, s), 0.00) 
_______________________________ Solución para producto Temperatura _______________________________
 Iteración: 0 
Estación: (E1, (t, s), 0.00)
Path: ['E1'] - Costo Acc: 0.00
Hijos: (E2, (t), 12.

# UCS

In [18]:
print("_______________________________ Solución UCS para producto Sanitario _______________________________")
if ucs_path_sanitaria:
    solucion_interactiva(ucs_path_sanitaria[0], "s")
print("_______________________________ Solución UCS para producto Temperatura _______________________________")
if ucs_path_temperatura:
    solucion_interactiva(ucs_path_temperatura[0], "t")

_______________________________ Solución UCS para producto Sanitario _______________________________
 Iteración: 0 
Estación: (E1, (t, s), 0.00)
Path: ['E1'] - Costo Acc: 0.00
Hijos: (E2, (t), 12.40) (E3, (t), 9.80) (E4, (t), 14.20) (E12, (s), 8.70) (E13, (s), 11.30) (E14, (s), 9.10) 
 Iteración: 1 
Estación: (E14, (s), 9.10)
Path: ['E1', 'E14'] - Costo Acc: 9.10
Hijos: (E19, (s), 8.30) (E20, (s), 9.90) (E16, (s), 10.80) 
 Iteración: 2 
Estación: (E20, (s), 9.90)
Path: ['E1', 'E14', 'E20'] - Costo Acc: 19.00
Hijos: (E21, (s), 7.20) (E23, (t, s), 0.00) 
_______________________________ Solución UCS para producto Temperatura _______________________________
 Iteración: 0 
Estación: (E1, (t, s), 0.00)
Path: ['E1'] - Costo Acc: 0.00
Hijos: (E2, (t), 12.40) (E3, (t), 9.80) (E4, (t), 14.20) (E12, (s), 8.70) (E13, (s), 11.30) (E14, (s), 9.10) 
 Iteración: 1 
Estación: (E3, (t), 9.80)
Path: ['E1', 'E3'] - Costo Acc: 9.80
Hijos: (E7, (t), 13.10) (E8, (t), 11.70) (E5, (t), 10.50) 
 Iteración: 2 
E

# BFS

In [19]:
print("_______________________________ Solución para producto Sanitario _______________________________")
solucion_interactiva((camino_bfs_sanitaria, costo_bfs_sanitaria)[0], "s")
print("_______________________________ Solución para producto Temperatura _______________________________")
solucion_interactiva((camino_bfs_temperatura, costo_bfs_temperatura)[0], "t")

_______________________________ Solución para producto Sanitario _______________________________
 Iteración: 0 
Estación: (E1, (t, s), 0.00)
Path: ['E1'] - Costo Acc: 0.00
Hijos: (E2, (t), 12.40) (E3, (t), 9.80) (E4, (t), 14.20) (E12, (s), 8.70) (E13, (s), 11.30) (E14, (s), 9.10) 
 Iteración: 1 
Estación: (E14, (s), 9.10)
Path: ['E1', 'E14'] - Costo Acc: 9.10
Hijos: (E19, (s), 8.30) (E20, (s), 9.90) (E16, (s), 10.80) 
 Iteración: 2 
Estación: (E20, (s), 9.90)
Path: ['E1', 'E14', 'E20'] - Costo Acc: 19.00
Hijos: (E21, (s), 7.20) (E23, (t, s), 0.00) 
_______________________________ Solución para producto Temperatura _______________________________
 Iteración: 0 
Estación: (E1, (t, s), 0.00)
Path: ['E1'] - Costo Acc: 0.00
Hijos: (E2, (t), 12.40) (E3, (t), 9.80) (E4, (t), 14.20) (E12, (s), 8.70) (E13, (s), 11.30) (E14, (s), 9.10) 
 Iteración: 1 
Estación: (E3, (t), 9.80)
Path: ['E1', 'E3'] - Costo Acc: 9.80
Hijos: (E7, (t), 13.10) (E8, (t), 11.70) (E5, (t), 10.50) 
 Iteración: 2 
Estación:

# 5. Experimentos Comparativos

| Algoritmo | Resultados | Comentarios

# 6. Análisis de Resultados

El algoritmo DFS tuve un peor desempeño en cuanto a que el camino generado no es óptimo y es más caro que las soluciones entragadas por BFS y por UCS. Esto era lo esperable dado que DFS es un algoritmo no informado y que el problema es relativamente pequeño no trivial.